In [0]:
from keras.datasets import mnist
from keras.layers import Activation,Reshape,Conv2D,Dense,Flatten,BatchNormalization,Dropout,UpSampling2D,Input,Conv2DTranspose,LeakyReLU,ReLU
from keras.optimizers import Adam
from keras.models import Model,Sequential
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [0]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

optimizer = Adam(0.0002, 0.5)


In [0]:
def build_generator():

        model = Sequential()

        model.add(Dense(7*7*128, input_dim=latent_dim))
        model.add(Reshape((7, 7, 128)))
        
        model.add(Conv2DTranspose(128, kernel_size=5, padding="same"))
        model.add(BatchNormalization(momentum=0.5))
        model.add(ReLU())
        
        
        model.add(Conv2DTranspose(64, kernel_size=5, padding="same",strides=2))
        model.add(BatchNormalization(momentum=0.5))
        model.add(ReLU())
        
        model.add(Conv2DTranspose(64, kernel_size=5, padding="same"))
        model.add(BatchNormalization(momentum=0.5))
        model.add(ReLU())
        
        model.add(Conv2DTranspose(1, kernel_size=5, padding="same",strides=2))
        model.add(BatchNormalization(momentum=0.5))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(latent_dim,))
        img = model(noise)

        return Model(noise, img)



In [0]:
def build_discriminator():

    model = Sequential()

    model.add(Conv2D(64, kernel_size=5, strides=2, input_shape=img_shape, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))


    model.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(256, kernel_size=5, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)



In [0]:
def combined():
        # The generator takes noise as input and generates imgs
        z = Input(shape=(latent_dim,))
        gentd_img = generator(z)

        # For the combined model we will only train the generator
        discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = discriminator(gentd_img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        combined = Model(z, valid)
        combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
        return combined

In [0]:
def train(epochs, batch_size=128, save_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        #  Train Discriminator

        # Select a random half of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # Sample noise and generate a batch of new images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)

        # Train the discriminator (real classified as ones and generated as zeros)
        
        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        #  Train Generator

        # Train the generator (wants discriminator to mistake images as real)
        g_loss = gan.train_on_batch(noise, valid)

        # Display the progress after every 200 epoch
        if epoch%200 == 0 :
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
        
        # If at save interval => save generated image samples
        if epoch % save_interval == 0:
            # Plot the progress
                
            save_imgs(epoch)



In [0]:
def save_imgs(epoch):
    r, c = 3, 3
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()



In [0]:
!mkdir  images/

In [0]:
%%time
generator =  build_generator()
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])    
gan = combined()
train(epochs=20001, batch_size=32, save_interval=1000)







Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 7, 7, 128)         409728    
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 128)         512       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 14, 14, 64)        204864    
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 64) 

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.794272, acc.: 37.50%] [G loss: 0.609590]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


200 [D loss: 0.007873, acc.: 100.00%] [G loss: 6.654810]
400 [D loss: 0.032928, acc.: 100.00%] [G loss: 5.900127]
600 [D loss: 0.052406, acc.: 96.88%] [G loss: 7.205486]
800 [D loss: 0.001697, acc.: 100.00%] [G loss: 10.051389]
1000 [D loss: 0.019079, acc.: 100.00%] [G loss: 6.585712]
1200 [D loss: 0.141725, acc.: 95.31%] [G loss: 3.802435]
1400 [D loss: 0.050706, acc.: 96.88%] [G loss: 10.477589]
1600 [D loss: 0.032284, acc.: 100.00%] [G loss: 5.375966]
1800 [D loss: 0.054577, acc.: 100.00%] [G loss: 6.003273]
2000 [D loss: 0.122860, acc.: 95.31%] [G loss: 9.223886]
2200 [D loss: 0.130520, acc.: 98.44%] [G loss: 6.168232]
2400 [D loss: 0.048609, acc.: 100.00%] [G loss: 5.348415]
2600 [D loss: 0.004656, acc.: 100.00%] [G loss: 6.999157]
2800 [D loss: 0.132426, acc.: 95.31%] [G loss: 5.258495]
3000 [D loss: 0.056960, acc.: 98.44%] [G loss: 7.778897]
3200 [D loss: 0.126331, acc.: 98.44%] [G loss: 4.849991]
3400 [D loss: 0.026229, acc.: 100.00%] [G loss: 4.945498]
3600 [D loss: 0.637745, 